# Aula 11 Transfer Learning

## How to Ensemble Models in Keras

In [16]:
import numpy as np;
import keras
import keras.backend as K;
from keras import optimizers, callbacks, regularizers;
from keras.regularizers import l2;
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten;
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
import os

In [17]:
def leCsv(nomeDir, nomeArq, nl=0, nc=0):
    
 st=os.path.join(nomeDir,nomeArq);
 arq=open(st,"rt"); lines=arq.readlines(); arq.close();
 n=len(lines)

 linhas_separadas=[]
    
 for linha in lines:
    
     linha=linha.strip('\n'); linha=linha.split(';');
     linhas_separadas.append(linha);
    
 ay=np.empty((n),dtype='float32');
 ax=np.empty((n,nl,nc,3),dtype='float32');
    
 for i in range(len(linhas_separadas)):
    
     if i%10==0: print("Lendo arquivo",i,linha[0])
     linha=linhas_separadas[i];
     img_path=os.path.join(nomeDir,linha[0])
     t = image.load_img(img_path, target_size=(nl,nc))
     x = image.img_to_array(t)
     x = np.expand_dims(x, axis=0)
     ax[i] = preprocess_input(x)
     ay[i] = np.float32(linha[1]); #0=m ou 1=f
 return ax, ay;

In [18]:
num_classes=2; nl=224; nc=224
diretorioBd="dados"
ax, ay = leCsv(diretorioBd,"treino.csv", nl=nl, nc=nc); #200 imagens
qx, qy = leCsv(diretorioBd,"teste.csv", nl=nl, nc=nc); #100 imagens
vx, vy = leCsv(diretorioBd,"valida.csv", nl=nl, nc=nc); #100 imagens
ay = keras.utils.to_categorical(ay, num_classes)
qy = keras.utils.to_categorical(qy, num_classes)
vy = keras.utils.to_categorical(vy, num_classes)

Lendo arquivo 0 078b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 20 026b.jpg
Lendo arquivo 30 010b.jpg
Lendo arquivo 40 174b.jpg
Lendo arquivo 50 057b.jpg
Lendo arquivo 60 028b.jpg
Lendo arquivo 70 020b.jpg
Lendo arquivo 80 166b.jpg
Lendo arquivo 90 053b.jpg
Lendo arquivo 100 097b.jpg
Lendo arquivo 110 114b.jpg
Lendo arquivo 120 163b.jpg
Lendo arquivo 130 042b.jpg
Lendo arquivo 140 104b.jpg
Lendo arquivo 150 055b.jpg
Lendo arquivo 160 149b.jpg
Lendo arquivo 170 187b.jpg
Lendo arquivo 180 158b.jpg
Lendo arquivo 190 030b.jpg
Lendo arquivo 0 199b.jpg
Lendo arquivo 10 064b.jpg
Lendo arquivo 20 087b.jpg
Lendo arquivo 30 070b.jpg
Lendo arquivo 40 162b.jpg
Lendo arquivo 50 024b.jpg
Lendo arquivo 60 197b.jpg
Lendo arquivo 70 154b.jpg
Lendo arquivo 80 132b.jpg
Lendo arquivo 90 068b.jpg
Lendo arquivo 0 098b.jpg
Lendo arquivo 10 036b.jpg
Lendo arquivo 20 127b.jpg
Lendo arquivo 30 061b.jpg
Lendo arquivo 40 151b.jpg
Lendo arquivo 50 041b.jpg
Lendo arquivo 60 043b.jpg
Lendo arquivo 70 049b.jpg
Lendo

In [19]:
models = []

for i in range(0, 6):
    
    models.append(load_model("resnet_transf"+str(i)+".h5"))

In [20]:
Y_preds=[]

for model in models:
    
    Y_preds.append(model.predict(qx))

In [21]:
ens_predictions = np.zeros([Y_preds[0].shape[0], Y_preds[0].shape[1]])

for coluna in range(0, Y_preds[0].shape[1]):
    
    for linha in range(0, Y_preds[0].shape[0]):
        
        ens_predictions[linha][coluna] = np.mean([Y_preds[0][linha][coluna], Y_preds[1][linha][coluna], Y_preds[2][linha][coluna], Y_preds[3][linha][coluna], Y_preds[4][linha][coluna], Y_preds[5][linha][coluna]])
        

In [22]:
classif = []

for linha in ens_predictions:
    
    classif.append(np.argmax(linha))

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
print("Acuracia de Teste: ", accuracy_score(classif, qy[:,1]));

Acuracia de Teste:  0.94


In [25]:
Y_preds=[]

for model in models:
    
    Y_preds.append(model.predict(vx))

In [26]:
ens_predictions  = np.zeros([Y_preds[0].shape[0], Y_preds[0].shape[1]])

for coluna in range(0, Y_preds[0].shape[1]):
    
    for linha in range(0, Y_preds[0].shape[0]):
        
        ens_predictions[linha][coluna] = np.mean([Y_preds[0][linha][coluna], Y_preds[1][linha][coluna], Y_preds[2][linha][coluna], Y_preds[3][linha][coluna], Y_preds[4][linha][coluna], Y_preds[5][linha][coluna]])
        

In [27]:
classif = []

for linha in ens_predictions:
    
    classif.append(np.argmax(linha))

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
print("Acuracia de Validação: ", accuracy_score(classif, vy[:,1]))

Acuracia de Validação:  0.95
